In [10]:
using JuMP, Cbc

nodes= 1:4
arcs=[(1,2), (1,3), 
    (2,3), (2,4), 
    (3,4)]
transsihpment = 2:3

b = [10,0,0,-10]
c = Dict(zip(arcs,[3,2,2,5,1]))
U = Dict(zip(arcs, [1000, 1000, 1000, 1000, 1000]))
L = Dict(zip(arcs, [0, 0, 0, 0, 0]))
u = Dict(zip(transsihpment, [6,6]))

Dict{Int64, Int64} with 2 entries:
  2 => 6
  3 => 6

In [13]:
m = Model()

@variable(m, L[a] <= x[a in arcs] <= U[a])

@objective(m, Min, sum(c[a]x[a] for a in arcs))

@constraint(m, flowbalance[i in nodes],
    sum(x[(i,j)] for j in nodes if (i,j) in arcs) - 
    sum(x[(j,i)] for j in nodes if (j,i) in arcs) 
    == b[i])

@constraint(m, transship_balance[i in transsihpment],
    sum(x[(j,i)] for j in nodes if (j,i) in arcs) 
    <= u[i])

set_optimizer(m, Cbc.Optimizer)
optimize!(m)

println("Mininum cost = ", round(objective_value(m),digits=1))
for a in arcs
    println("flow on arc ", a, "= ", round(value(x[a]),digits=1))
end

Mininum cost = 50.0
flow on arc (1, 2)= 4.0
flow on arc (1, 3)= 6.0
flow on arc (2, 3)= 0.0
flow on arc (2, 4)= 4.0
flow on arc (3, 4)= 6.0
Presolve 3 (-3) rows, 3 (-2) columns and 8 (-5) elements
0  Obj 41.499999 Primal inf 4.0999973 (3)
3  Obj 50
Optimal - objective value 50
After Postsolve, objective 50, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 50 - 3 iterations time 0.002, Presolve 0.00


In [23]:
nmonths = 6
cost = [12, 14, 21, 22, 15, 13]
backlogcost = 6
demand = [100, 132, 137, 153, 144, 116]

m = Model()

@variable(m, x[1:nmonths] >= 0)
@variable(m, b[1:nmonths] >= 0)

@objective(m, Min, sum(cost[t] * x[t] + backlogcost * b[t] for t in 1:nmonths))

@constraint(m, initial_demand, x[1] + b[1] >= demand[1])
@constraint(m, inventory_balance[t in 2:nmonths], x[t] + b[t] >= b[t-1] + demand[t])
@constraint(m, final_inventory, b[nmonths] == 0)

set_optimizer(m, Cbc.Optimizer)
optimize!(m)
solution_summary(m, verbose=true)

Presolve 6 (-1) rows, 11 (-1) columns and 16 (-2) elements
0  Obj 1506.7 Primal inf 666.09999 (6)
7  Obj 12806
Optimal - objective value 12806
After Postsolve, objective 12806, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 12806 - 7 iterations time 0.002, Presolve 0.00


* Solver : COIN Branch-and-Cut (Cbc)

* Status
  Termination status : OPTIMAL
  Primal status      : FEASIBLE_POINT
  Dual status        : NO_SOLUTION
  Result count       : 1
  Has duals          : false
  Message from the solver:
  "Cbc_status          = finished - check isProvenOptimal or isProvenInfeasible to see if solution found (or check value of best solution)
Cbc_secondaryStatus = unset (status_ will also be -1)
"

* Candidate solution
  Objective value      : 12806.0
  Objective bound      : 12806.0
  Primal solution :
    b[1] : 0.0
    b[2] : 0.0
    b[3] : 0.0
    b[4] : 153.0
    b[5] : 0.0
    b[6] : 0.0
    x[1] : 100.0
    x[2] : 132.0
    x[3] : 137.0
    x[4] : 0.0
    x[5] : 297.0
    x[6] : 116.0

* Work counters
  Solve time (sec)   : 0.00000
